# Tensorfow 모델에 pruning 적용하는 방법

1. 개요
    - pruning 알고리즘은 'graudal pruning' 방식 적용: 목표 sparisty값에 도달할때까지 진행하며 pruning 속도를 완만하게 늦춘다
    - 실험: (1) MNIST tf.keras모델 학습
           (2) pruning API를 적용하면서 fine-tune(재학습)

2. 진행 방식

![image.png](https://k.kakaocdn.net/dn/c3vsry/btqzpWxXLf4/B5dke1NkTREakQsGiSmd50/img.png)

3. pruning 모델 생성
    - pruning 대상이 되는 layer에 mask와 threshold 변수를 추가한다
    - mask는 해당 layer의 weight tensor와 shape이 일하며 forward execution에 적용할 weight을 결정하는 역할을 수행함
    - apply_mask()함수를 이용하여 layer의 weight tensor를 wrapping 함으로써, mask와 threshold가 추가된 convolutional layer를 생성한다
    

4. 방법
    - 설정하는 부분
        (1) pip install -q tensorflow-model-optimization
            #머신러닝 모델 최적화 도구
            
       

### tensorflow api 사용 예제

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

* tf.keras는 케라스 API의 tensorflow 구현이다.
* tf.keras는 케라스 API와 호환되는 어떤 코드라도 실행시킬 수 있다.

### train model for MNIST without pruning

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)


##### Sequential 모델
- 케라스에서는 layer를 조합하여 모델을 만든다.
- 가장 흔한 모델 구조는 층을 차례대로 쌓은 keras.Sequential 모델이다.

##### 훈련과 평가
- model.compile() : 위에서 모델을 구성한 후 compile함수를 호출하여 학습 과정 설정한다
    위 함수에는 중요한 3가지 매개변수가 있다
    
    optimizer = 훈련과정을 설정(ex_ 'adam', 'sgd'와 같은 문자열로 지정 가능)
    loss = 최적화 과정에서 최소화될 손실 함수를 설정
    metrics = 훈련을 모니터링하기 위해 사용된다
    
    
- model.fit() : 모델은 fit함수를 통해서 훈련 data를 학습한다

    위 함수에는 중요한 3가지 매개변수가 있다
    epochs = 훈련은 epoch로 구성되어 있다. 한 epoch에는 전체 입력 데이터를 한번 순회하는 것이다
    batch_size = 모델은 data르ㄹ 작은 batch로 나누고 훈련 과정에서 이 배치를 순회한다.
    

##### 분류에 쓰이는 손실 (MNIST 문자 인식)
    1. binary cross-entropy
    2. categorical cross-entropy
    3. sparse categorical cross-entropy
    4. hinge 
    5. squared Hinge
    6. Categorical Hinge

##### Otimizer종류
    1. GD(Gradient Descent)
        모든 자료를 다 검토해서 내 위치의 기울기를 계산해서 갈 방향 찾는다
            --> 경사를 따라 내려가면서 W를 update시킨다.
        
    2. SGD(Stochastic gradient decent)
        전부 다 검토하면 시간이 오래 걸리니까 조금만 보고 빨리 판단한다
            --> full-batch가 아닌 mini batch로 학습을 진행하는 것
    
    3. Momentum
        스텝 계산해서 움직인 후, 내려오던 관성 방향으로 진행
            --> SGD에 MOMENTUM개념을 추가한 것이다.
            --> 현재 batch로만 학습하는 것이 아니라 이전의 batch학습결과도 반영한다.
      
    4. Adagrad
        안가본곳은 빠르게 훓고 많이 가본곳은 잘 아니까 갈수록 보폭을 줄여 세밀히 탐색
        --> 학습을 통해 크게 변동이 있었던 가중치에 대해서는 학습률 감소, 아직 가중치의 변동이 별로 없었던 가중치는 학습률 증가
        --> 기존 notation에서 h가 추가되었는데 h는 가중치 기울기 제곱들을 더해간다.따라서 가중치 값에 많은 변동이 있었던 가중치는 점점 학습률 감소 시킨다.
        
    5. RMSProp
        보폭을 줄이는건 좋은데 이전 맥락 상황 봐가면서 하자
        --> Adagrad는 복잡한 다차원 곡면 funtion에서는 학습률이 0에 수렴 가능성 있음. 따라서 이를 보완한것이 RMSProp이다.
        --> 가중치 기울기를 단순 누적시키는 것이 아니라 최신 기울기들이 더 반영되도록 한다.
        
    6. Adam
        RMSProp + Momentum
        --> momentum은 v와 h가 처음에 0으로 초기화되면 W가 0으로 biased가된다.
        -->이런 문제점을 보완하기 위해서 Adam이 나옴

#### Evaluate baseline test accuracy and save the model for later usage.

In [1]:
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)


NameError: name 'model' is not defined

##### 평가 및 모델 저장
* model.evaluate() : 주어진 data로 추론 모드의 손실이나 지표를 평가한다.
                 return 값으로 tuple로 나오는데 2번째 값이 정확도 값이다

* models.save_model(model, filepath, include_optimizer, save_format) : 
            모델을 저장하는데, HDF5를 사용하기 위해 h5로 설정함
                        

tf.keras.models.save_model(model, './', include_optimizer=False,save_format='h5')

* tempfile.mkstemp('.h5'): 임시저장공간으로 사용할 수 있는 file-like객체를 반환하고, 파일은 mkstemp()를 이용해 생성된다. 

### Fine-tune pre-trained model with pruning

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()


#### 전체 모델에 pruning을 적용시키고 model summary에 대한 내용 보기

- 머신러닝 모델 최적화 도구인 tensorflow_model_optimization import한다
    +  tfmot.sparsity.keras.prune_low_magnitude(): 훈련 하면서 pruning 할 keras.layer 또는 모델을 수정한다
    
    
        -  이 funtion은 훈련중에 layer의 가중치를 pruning 한다(layer의 weight값을 sparsify(0으로 만든다)한다)
            (ex) sparsity=50% -> layer weight의 50%가 0이 된다.
            
        - 이 funtion은 단일 keras layer, list of keras layer, Sequential or Functional keras model을 받아들여 처리한다.
        

### prune a model

1. pruning_params = {'pruning_schedule':tfmot.sparsity.keras.PolynomialDecay()}
                
    **tfmot.sparsity.keras.PolynomialDecay(initial_sparsity, final_sparsity, begin_step, end_step, power=3, frequency=100)**


- 'initial_sparsity', 'final_sparsity': model pruning을 초기 sparsity에서 시작하여 훈련하면서 목표 sparsity=final_sparsity에 도달할때 까지 pruning을 진행한다.

- 'begin_step', 'end_step' : Step at which to begin pruning, Step at which to end pruning. 

- 'power' : sparsity funtion에서 사용될 지수

- 'frequency': frequency step 마다 pruning 적용


2. **pruning_schedule**: 각 단계에서 레이어의 가중치를 제거해야하는지 여부와 제거해야하는 희소성 (%)을 알려 각 훈련 단계에서 제거를 제어합니다.

current_sparsity = final_sparsity + (initial_sparsity - final_sparsity) * (1 - (step - begin_step)/(end_step - begin_step)) ^ exponent

3. prune a model 

     **prune_low_magnitude(model, **pruning_params)**
        

    -각 layer마다 pruning 구성을 어떻게 다르게 할것인가에 대한 세부사항 정의


4. model_for_pruning.summary()

        - trainable params는 total params의 반값이다 -> sparisty를 50%로 잡았기 때문이다.
    
        - 모델 요약은 총 모델에 사용되는 parmameter의 개수이다!


5. model.compile(): 학습 과정 설정한다
    - prune_low_magnitude 함수는 optimizer과정이 없다.
    

#### Train and evaluate the model against baseline

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)


6. fine tune with pruning

    6.1 **tfmot.sparsity.keras.UpdatePruningStep()**
        
        - Keras callback which updates pruning wrappers with the optimizer step.
        
        - fine-tuning: 기존에 학습되어 있는 모델을 기반으로 아키텍처를 새로운 목적에 맞게 변형하고 이미 학습된 모델 weight로 부터 학습을 update하는 방법
        
        - wrapper: 가장 이상적인 변수의 조합을 찾는 방식
        
        - 즉, training의 optimizer 단계 대신 pruning wrapper 단계를 넣어 update한다.
        
        

    6.2 tfmot.sparsity.keras.PruningSummaries()
    
        - 주어진 반복 단계에서 희소성 (%) 및 임계 값을 기록합니다(tensorboard에 넣기 위한 작업)
        

7. training

    model_for_pruning.fit(.., callbacks=callbacks)

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

- pre-trained model의 정확도와 pruned model의 정확도 비교

### Create 3x smaller models from pruning

* pruning의 compression benefits를 얻기 위해서는 

    tfmot.sparsity.keras.strip_pruning() + standard compression algorithm (e.g. via gzip)
    
    1. tfmot.sparsity.keras.strip_pruning(model)
    
        - model의 pruning wrapper를 제거한다.
        
        - 즉, 모델이 pruning된 후에 이 function을 사용하여 sparse weight로 원래 모델로 복원한다.
        
        - The exported_model and the orig_model share the same structure.

In [ ]:
## 이 과정은 tensorflow와 호환되는 model로 만들기!

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

In [2]:
## 위 model을 가지고 TFLite와 호환되는 model로 만들기!

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


NameError: name 'tf' is not defined